In [16]:
import json, copy, re
import pandas as pd

In [17]:
with open("uca.json", 'r') as file:
    data = json.load(file)
    
profiles = copy.deepcopy(data)
articles_data = copy.deepcopy(data)

# Transform profile Data

#### Transform the `Article` and `Spciality`

In [18]:
for item in profiles:
    item["Spciality"] = ", ".join(item["Spciality"])
    item["Article"] = len(item["Article"])

#### Add 'Faculty'

In [19]:
class Transform:
    def __init__(self, to, test) -> None:
        self.value = to
        self.test = test


def test(e, search):
    return any(s.lower() in e.lower() for s in search)


def transform_to_fstg(e):
    return test(e, ["FST", "FSTG", "Faculty of Sciences and Techniques", "Faculté des Sciences et Techniques"])


def transform_to_ensa(e):
    return test(e, ["ENSA"])


def transform_to_fssm(e):
    return test(e, ["FSSM", "Semlalia", "Faculty of Sciences"])


def transform_to_other(e):
    # return e == "Université Cadi Ayyad" or e == "CADI AYYAD UNIVERSITY"
    return test(e, ["Université Cadi Ayyad",
                    "Cadi Ayyad University",
                    "Cadi  Ayyad",
                    "CADI AYYAD UNIVERSITY"
                    ])

faculties_transform = [
    Transform("FSTG", transform_to_fstg),
    Transform("ENSA", transform_to_ensa),
    Transform("FSSM", transform_to_fssm),
    Transform("Other", transform_to_other),
]


def transform_faculties(data):
    for ind, item in enumerate(data):
        for transform in faculties_transform:
            if transform.test(item.get("ASS - Unevercity", "")):
                data[ind]["faculty"] = transform.value
                break
            # else: data[ind]["faculty"] = item.get("ASS - Unevercity")
            else: data[ind]["faculty"] = "None"

In [20]:
# from faculty_transform import transform_faculties
profiles = copy.deepcopy(profiles)
transform_faculties(profiles)

In [21]:
# for prof in profiles:
#     print(prof["faculty"])


In [22]:
profiles_df = pd.DataFrame(profiles)
profiles_df

,Nom et Prénom,ASS - Unevercity,Spciality,H indice global,Citations global,H indice_I10 global,H indice depuis 2018,Citations depuis 2018,H indice_I10 depuis 2018,Article,faculty
0,salah er-raki,"ProcEDE- FST de Marrakech,Université Cadi Ayyad","physique, irrigation, Agricultural Sciences, r...",37,5231,69,34,3308,63,20,FSTG
1,Abdelkader Outzourhit,Université Cadi Ayyad,"Energies renouvelables, PV, Stockage",29,3973,105,22,1899,66,20,Other
2,Jamal Ezzahar,"MISCOM, ENSA Safi,Université Cadi Ayyad Marrakech","Land-atmosphere interactions, Soil Moisture, R...",28,3703,46,26,2399,41,20,ENSA
3,ouazzani naaila,"professeur à l'université Cadi Ayyad, Marrakech",Wastewater treatment and reuse,31,3549,72,26,2502,60,20,Other
4,Saïd Khabba,Professeur de l'Université Cadi Ayyad,"Ressources en eau, Besoin en eau des cultures,...",32,3474,65,28,2490,53,20,Other
5,ABDELGHANI BELLOUQUID,"professeur de mathématique,université cadi ayyad",modélisation des systèmes complexes et analyse...,28,3362,43,16,1571,23,20,Other
6,Soulaimani Abderrahmane,"Professor,Cadi Ayyad University, Marrakech, Mo...",Tectonic and Regional Geology,29,3322,52,25,2041,44,20,Other
7,GHOULAM Cherki,"Faculty of Sciences and Techniques,Cadi Ayyad ...",Agrophysiologie,29,3213,49,22,1858,43,20,FSTG
8,Yedir Ouhdouch,professeur de microbiologieUniversité Cadi Ayyad,"Actinobactéries, biotechnologie microbienne, s...",25,3007,46,20,1875,40,20,Other
9,El Modafar cherkaoui,"Professeur de Biologie,Université Cadi Ayyad",Biotechnologies végétles,31,2963,77,25,2052,58,20,Other


In [24]:
profiles_df.to_csv("./generated/profiles-v2.csv", index=False)